In [1]:
from qutip import *
import cmath
import math
import matplotlib.pyplot as plt
import numpy as np

In [2]:
nqubits = 3

init = []
for _ in range(nqubits):
    init.append(basis(2,0))

inital_state = tensor(init)

In [3]:
inital_state

Quantum object: dims=[[2, 2, 2], [1, 1, 1]], shape=(8, 1), type='ket', dtype=Dense
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]

In [4]:
# 2^n dim ideentity operator, n: number of qubits
identity_list = []
for _ in range(nqubits):
    identity_list.append(qeye(2))
Qeye = tensor(identity_list)

In [5]:
Qeye

Quantum object: dims=[[2, 2, 2], [2, 2, 2]], shape=(8, 8), type='oper', dtype=Dia, isherm=True
Qobj data =
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]

In [6]:
# X, Y, Z operator

def Xop(m: int, nqubits: int) -> object:
    """
    Constructs the Pauli X operator acting on the m-th qubit 
    in a quantum system of n total qubits.

    Args:
        m (int): The index of the target qubit (0-based).
        nqubits (int): The total number of qubits in the system.

    Returns:
        object: The tensor product operator with the Pauli X operator 
                applied to the m-th qubit and identity operators on 
                all other qubits.
    """

    operators = []

    for n in range(nqubits):
        if n == m:
            operators.append(sigmax())
        else:
            operators.append(qeye(2))

    return tensor(operators)

def Yop(m: int, nqubits: int) -> object:
    """
    Constructs the Pauli Z operator acting on the m-th qubit 
    in a quantum system of n total qubits.

    Args:
        m (int): The index of the target qubit (0-based).
        nqubits (int): The total number of qubits in the system.

    Returns:
        object: The tensor product operator with the Pauli X operator 
                applied to the m-th qubit and identity operators on 
                all other qubits.
    """

    operators = []

    for n in range(nqubits):
        if n == m:
            operators.append(sigmay())
        else:
            operators.append(qeye(2))

    return tensor(operators)

def Zop(m: int, nqubits: int) -> object:
    """
    Constructs the Pauli Z operator acting on the m-th qubit 
    in a quantum system of n total qubits.

    Args:
        m (int): The index of the target qubit (0-based).
        nqubits (int): The total number of qubits in the system.

    Returns:
        object: The tensor product operator with the Pauli X operator 
                applied to the m-th qubit and identity operators on 
                all other qubits.
    """

    operators = []

    for n in range(nqubits):
        if n == m:
            operators.append(sigmaz())
        else:
            operators.append(qeye(2))

    return tensor(operators)
    

In [7]:
# Hamiltonian

def construct_hami(nqubits:int, v: float, hx: float, hz: float) -> object:

    hamiltonian = []
    for n in range(nqubits-1):
        hamiltonian.append(v/4 * Zop(n, nqubits)*Zop(n+1, nqubits))
    for n in range(nqubits):
        hamiltonian.append(hx * Xop(n, nqubits))
        hamiltonian.append(hz * Zop(n, nqubits))
    
    # Initialize the Hamiltonian as a zero operator
    Hamil = 0 * Zop(0, nqubits)

    for term in hamiltonian:
        Hamil += term
    
    return Hamil

In [8]:
hamiltonian = construct_hami(nqubits= nqubits, v=1.0, hx=1.0, hz=0)
hamiltonian

Quantum object: dims=[[2, 2, 2], [2, 2, 2]], shape=(8, 8), type='oper', dtype=CSR, isherm=True
Qobj data =
[[ 0.5  1.   1.   0.   1.   0.   0.   0. ]
 [ 1.   0.   0.   1.   0.   1.   0.   0. ]
 [ 1.   0.  -0.5  1.   0.   0.   1.   0. ]
 [ 0.   1.   1.   0.   0.   0.   0.   1. ]
 [ 1.   0.   0.   0.   0.   1.   1.   0. ]
 [ 0.   1.   0.   0.   1.  -0.5  0.   1. ]
 [ 0.   0.   1.   0.   1.   0.   0.   1. ]
 [ 0.   0.   0.   1.   0.   1.   1.   0.5]]

In [17]:
type(hamiltonian)

qutip.core.qobj.Qobj

In [9]:
eigen_vals, eigen_vecs = hamiltonian.eigenstates()
eigen_vals

array([-3.03124812, -1.36815091, -1.        , -0.66309722,  0.66309722,
        1.        ,  1.36815091,  3.03124812])

In [10]:
ground_state_energy = np.min(eigen_vals)
print(f"The min eigen value is: {ground_state_energy}")

The min eigen value is: -3.031248122331948


In [11]:
def filter_function(a: float, t: float)->float:
    return np.sqrt(a/np.pi) * np.exp(-a*t*t)

In [12]:
def pauli_noise(rho: object, nqubits: int, px: float, py: float, pz: float) -> object:
    """
    Applies Pauli noise to a quantum state (density matrix).

    Parameters:
    rho (object): Density matrix of the quantum state.
    nqubits (int): Number of qubits in the system.
    px (float): Probability of applying X (bit-flip) noise.
    py (float): Probability of applying Y (bit-phase-flip) noise.
    pz (float): Probability of applying Z (phase-flip) noise.

    Returns:
    object: Noisy density matrix.
    """
    # Identity term (no noise)
    rho_prime = (1 - px - py - pz) * rho

    # Apply X noise to each qubit
    for n in range(nqubits):
        rho_prime += px * Xop(n, nqubits) * rho * Xop(n, nqubits).dag()

    # Apply Y noise to each qubit
    for n in range(nqubits):
        rho_prime += py * Yop(n, nqubits) * rho * Yop(n, nqubits).dag()

    # Apply Z noise to each qubit
    for n in range(nqubits):
        rho_prime += pz * Zop(n, nqubits) * rho * Zop(n, nqubits).dag()

    return rho_prime


In [18]:
def trotter(hamiltonian: object, time: float, steps: int) -> object:

    trotter_term = Qeye
    
    for n in range(len(hamiltonian)):
        trotter_term = trotter_term * (-1j * hamiltonian[n]*t/steps).exmp()

    
    trotter_hamiltonian = trotter_term**steps

    return trotter_hamiltonian


def control_unitary(hamiltonian: object, time: float) -> object:

    u_gate = (-1j*hamiltonian*time).expm()

    control_u_gate = tensor(basis(2,0)*basis(2,0).dag(), Qeye) + tensor(basis(2,1)*basis(2,1).dag(), u_gate)

    return control_u_gate

def control_trotter(hamiltonian: object, time: float, steps: int) -> object:

    trotter_hamiltonian = trotter(hamiltonian, time, steps)

    control_trotter = tensor(basis(2,0)*basis(2,0).dag(), Qeye) + tensor( basis(2,1)* basis(2,1).dag, trotter_hamiltonian)


def ising_trotter(nqubits: int,
                  rho: object,
                  steps: int,
                  time: float,
                  v: float,
                  h: float,
                  px: float,
                  py: float,
                  pz: float
                ) -> object:
    

    state = rho

    for n in range(num - 1):

        ctrl_uni = tensor(basis(2,0)*basis(2,0).dag(), Qeye()) + tensor(basis(2,1)*basis(2,1).dag(), (-1j * (v/4) * Zop(n, nqubits) * Zop(n+1, nqubits) * time/steps ).exmp())

        state = ctrl_uni * state * ctrl_uni.dag()

        noisy_state = pauli_noise(state, nqubits, px, py, pz)
    
    for n in range(nqubits):

        ctrl_uni = tensor(basis(2,0)*basis(2,0).dag(), Qeye()) + tensor(basis(2,1)*basis(2,1).dag(), (-1j * (h) * Xop(n, nqubits) * time/steps ).exmp())
 
        noisy_state = ctrl_uni * noisy_state * ctrl_uni.dag()

    return state

In [21]:
import random
def hadamard_test(nqubits: int,
                  delt: float,
                  steps: int,
                  tw: float,
                  shots: int,
                  px: float,
                  py: float,
                  pz: float) -> object:
    
    # Ancilla state
    ancilla_state = (1/np.sqrt(2)) * (basis(2,0)+basis(2,1))
    ancilla_state_dm = ket2dm(ancilla_state)

    # System state
    sys_init_state_dm = init

    # Tensor product state of ancilla and system
    prod_state = tensor(ancilla_state_dm, ket2dm(sys_init_state_dm))

    time = []

    EX, EY = [], []

    for t in range(-tw, tw + 1):
        t1 = delt*t
        time.append(t1)

        state = prod_state

        for _ in range(steps):
            state = ising_trotter(nqubits, state, steps, t1, 1, 0, 0, 0, 0)

        expectX = (tensor(sigmax, Qeye) * state).tr()
        expectY = (tensor(sigmay, Qeye) * state).tr()

        varx = 1 - expectX**2
        vary = 1 - expectY**2

        randx = random.gauss(mu=expectX, sigma=np.sqrt(varx / shots))
        randy = random.gauss(mu=expectY, sigma=np.sqrt(vary / shots))

        EX.append(randx)
        EY.append(randy)

        # Sweep energy
        a = 0.001
        delE = 0.1
        Range = 100
        Val,  Energy = [], []



